In [93]:
import tensorflow as tf
import numpy as np
import base64
import keras
import os
import sys
from PIL import Image
import skimage.data
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt

In [36]:
# Compress images

def resize(folder, fileName):
    filePath = os.path.join(folder, fileName)
    im = Image.open(filePath)
    w, h  = im.size
    newIm = im.resize((64, 64))
    newIm.save(filePath)

def bulkResize(imageFolder):
    imgExts = ["png", "jpg"]
    for path, dirs, files in os.walk(imageFolder):
        for fileName in files:
            ext = fileName[-3:].lower()
            if ext not in imgExts:
                continue
            resize(path, fileName)

#imageFolder="/Users/Chrsitine/Desktop/Conjurers/images/yellow_oreos/" 
bulkResize(imageFolder)
print("Done resizing")

In [67]:
# 0-indexing with one hot encoding is used for class labels:
idx_to_label = {0: "Coke zero", 1: "Oreo thins (original)", 2: "Cadbury chocolate fingers",
               3: "Diet coke", 4: "Hershey's drops", 5: "m&ms", 6: "Sun chips", 
               7: "Sour cream and onion Pringles", 8: "Coke (original)", 9: "Golden Oreo thins"}

In [105]:
# Load training and testing data 
x_train = []
y_train = []
x_test = []
y_test = []
train_images = []
test_images = []
labels = []

imgExts = ["png", "jpg"]
counter = -1
for path, dirs, files in os.walk("/Users/Chrsitine/Desktop/Conjurers/images/"):
    for fileName in files:
        ext = fileName[-3:].lower()
        if ext not in imgExts:
            continue
        else:
            filePath = os.path.join(path, fileName)
            img = Image.open(filePath)
            img_array = np.array(img.getdata(), np.uint8).reshape(img.size[1], img.size[0], 3)
            x_train.append(img_array)
            train_images.append(skimage.data.imread(filePath))
            y_train.append(counter)
            label = np.zeros(10)
            label[counter] = 1
            #y_train.append(label)
    counter += 1

imgExts = ["png", "jpg"]
counter = -1
for path, dirs, files in os.walk("/Users/Chrsitine/Desktop/Conjurers/test_images/"):
    for fileName in files:
        ext = fileName[-3:].lower()
        if ext not in imgExts:
            continue
        else:
            filePath = os.path.join(path, fileName)
            img = Image.open(filePath)
            img_array = np.array(img.getdata(), np.uint8).reshape(img.size[1], img.size[0], 3)
            x_test.append(img_array)
            test_images.append(skimage.data.imread(filePath))
            label = np.zeros(10)
            label[counter] = 1
            y_test.append(counter)
    counter += 1

x_train = np.array(x_train)
y_train = np.array(y_train)
print(x_train.shape)
print(y_train.shape)

(550, 64, 64, 3)
(550,)


In [97]:
def display_images_and_labels(images, labels):
    """Display the first image of each label."""
    unique_labels = set(labels)
    plt.figure(figsize=(15, 15))
    i = 1
    for label in unique_labels:
        # Pick the first image for each label.
        image = images[labels.index(label)]
        plt.subplot(2, 5, i)  # A grid of 8 rows x 8 columns
        plt.axis('off')
        plt.title("Label {0} ({1})".format(label, labels.count(label)))
        i += 1
        _ = plt.imshow(image)
    plt.show()

display_images_and_labels(images, labels)

In [99]:
# Create a graph to hold the model.
graph = tf.Graph()

# Create model in the graph.
with graph.as_default():
    # Placeholders for inputs and labels.
    images_ph = tf.placeholder(tf.float32, [None, 64, 64, 3])
    labels_ph = tf.placeholder(tf.int32, [None])

    # Flatten input from: [None, height, width, channels]
    # To: [None, height * width * channels] == [None, 3072]
    images_flat = tf.contrib.layers.flatten(images_ph)

    # Fully connected layer. 
    # Generates logits of size [None, 62]
    logits = tf.contrib.layers.fully_connected(images_flat, 62, tf.nn.relu)

    # Convert logits to label indexes (int).
    # Shape [None], which is a 1D vector of length == batch_size.
    predicted_labels = tf.argmax(logits, 1)

    # Define the loss function. 
    # Cross-entropy is a good choice for classification.
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels_ph))

    # Create training op.
    train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

    # And, finally, an initialization op to execute before training.
    # TODO: rename to tf.global_variables_initializer() on TF 0.12.
    init = tf.global_variables_initializer()

print("images_flat: ", images_flat)
print("logits: ", logits)
print("loss: ", loss)
print("predicted_labels: ", predicted_labels)

images_flat:  Tensor("Flatten/flatten/Reshape:0", shape=(?, 12288), dtype=float32)
logits:  Tensor("fully_connected/Relu:0", shape=(?, 62), dtype=float32)
loss:  Tensor("Mean:0", shape=(), dtype=float32)
predicted_labels:  Tensor("ArgMax:0", shape=(?,), dtype=int64)


In [87]:
# Create a session to run the graph we created.
session = tf.Session(graph=graph)

# First step is always to initialize all variables. 
# We don't care about the return value, though. It's None.
_ = session.run([init])

In [89]:
for i in range(100):
    _, loss_value = session.run([train, loss], 
                                feed_dict={images_ph: x_train, labels_ph: labels})
    if i % 10 == 0:
        print("Loss: ", loss_value)

Loss:  4.12714
Loss:  4.12714
Loss:  4.12714
Loss:  4.12714
Loss:  4.12714
Loss:  4.12714
Loss:  4.12714
Loss:  4.12714
Loss:  4.12714
Loss:  4.12714


In [106]:
# Run predictions against the full test set.
predicted = session.run([predicted_labels], 
                        feed_dict={images_ph: test_images})[0]
# Calculate how many matches we got.
match_count = sum([int(y == y_) for y, y_ in zip(y_test, predicted)])
accuracy = match_count / len(y_test)
print("Accuracy: {:.3f}".format(accuracy))

TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("Placeholder:0", shape=(?, 64, 64, 3), dtype=float32) is not an element of this graph.

In [3]:
# Save trained model
model.save("trained_model1.hdf5")
model.save_weights("trained_weights1.hdf5")
print("Saved trained model and weights")